In [76]:
import pandas as pd
import duckdb as db
import numpy as np

## Question 1
What's the version of Pandas that you installed?

In [43]:
pd.__version__

'2.1.0'

In [44]:
! wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2023-09-17 19:50:42--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
Unable to establish SSL connection.


In [45]:
df = pd.read_csv("housing.csv")

In [46]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [48]:
db.sql("""
select * from df limit 3
""")

┌───────────┬──────────┬────────────────────┬─────────────┬───┬───────────────┬────────────────────┬─────────────────┐
│ longitude │ latitude │ housing_median_age │ total_rooms │ … │ median_income │ median_house_value │ ocean_proximity │
│  double   │  double  │       double       │   double    │   │    double     │       double       │     varchar     │
├───────────┼──────────┼────────────────────┼─────────────┼───┼───────────────┼────────────────────┼─────────────────┤
│   -122.23 │    37.88 │               41.0 │       880.0 │ … │        8.3252 │           452600.0 │ NEAR BAY        │
│   -122.22 │    37.86 │               21.0 │      7099.0 │ … │        8.3014 │           358500.0 │ NEAR BAY        │
│   -122.24 │    37.85 │               52.0 │      1467.0 │ … │        7.2574 │           352100.0 │ NEAR BAY        │
├───────────┴──────────┴────────────────────┴─────────────┴───┴───────────────┴────────────────────┴─────────────────┤
│ 3 rows                                        

## Question 2
How many columns are in the dataset?

    10
    6560
    10989
    20640

In [49]:
df.shape

(20640, 10)

## Question 3
Which columns in the dataset have missing values?

    total_rooms
    total_bedrooms
    both of the above
    no empty columns in the dataset

In [50]:
db.sql("""
       select
           count(1)
       from df
       where total_rooms is null
""")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│        0 │
└──────────┘

In [51]:
df.loc[df["total_rooms"].isnull()].shape

(0, 10)

In [52]:
db.sql("""
       select
           count(1)
       from df
       where total_bedrooms is null
""")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│      207 │
└──────────┘

In [53]:
df.loc[df["total_bedrooms"].isnull()].shape

(207, 10)

## Question 4
How many unique values does the ocean_proximity column have?

    3
    5
    7
    9

In [54]:
db.sql("""
    select
        count(distinct ocean_proximity)
    from df
""")

┌─────────────────────────────────┐
│ count(DISTINCT ocean_proximity) │
│              int64              │
├─────────────────────────────────┤
│                               5 │
└─────────────────────────────────┘

## Question 5
What's the average value of the median_house_value for the houses located near the bay?

    49433
    124805
    259212
    380440

In [55]:
db.sql("""
    select
        distinct ocean_proximity
    from df
""")

┌─────────────────┐
│ ocean_proximity │
│     varchar     │
├─────────────────┤
│ NEAR BAY        │
│ <1H OCEAN       │
│ INLAND          │
│ NEAR OCEAN      │
│ ISLAND          │
└─────────────────┘

In [56]:
db.sql("""
    select
        avg(median_house_value)
    from df
    where ocean_proximity = 'NEAR BAY'
""")

┌─────────────────────────┐
│ avg(median_house_value) │
│         double          │
├─────────────────────────┤
│      259212.31179039303 │
└─────────────────────────┘

## Question 6

1. Calculate the average of total_bedrooms column in the dataset.
2. Use the fillna method to fill the missing values in total_bedrooms with the mean value from the previous step.
3. Now, calculate the average of total_bedrooms again.
4. Has it changed?
5. Has it changed?

> Hint: take into account only 3 digits after the decimal point.

    Yes
    No

In [57]:
db.sql("""
    select
        avg(total_bedrooms),
        round(avg(total_bedrooms), 3)
    from df
""")

┌─────────────────────┬───────────────────────────────┐
│ avg(total_bedrooms) │ round(avg(total_bedrooms), 3) │
│       double        │            double             │
├─────────────────────┼───────────────────────────────┤
│   537.8705525375618 │                       537.871 │
└─────────────────────┴───────────────────────────────┘

In [58]:
db.sql("""
    select
        count(1)
    from df
    where total_bedrooms is null
""")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│      207 │
└──────────┘

In [59]:
df["total_bedrooms"].fillna(537.871, inplace=True)

In [60]:
db.sql("""
    select
        avg(total_bedrooms),
        round(avg(total_bedrooms), 3)
    from df
""")

┌─────────────────────┬───────────────────────────────┐
│ avg(total_bedrooms) │ round(avg(total_bedrooms), 3) │
│       double        │            double             │
├─────────────────────┼───────────────────────────────┤
│   537.8705570251923 │                       537.871 │
└─────────────────────┴───────────────────────────────┘

In [61]:
db.sql("""
    select
        count(1)
    from df
    where total_bedrooms is null
""")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│        0 │
└──────────┘

Answer: No

## Question 7
1. Select all the options located on islands.
2. Select only columns housing_median_age, total_rooms, total_bedrooms.
3. Get the underlying NumPy array. Let's call it X.
4. Compute matrix-matrix multiplication between the transpose of X and X. To get the transpose, use X.T. Let's call the result XTX.
5. Compute the inverse of XTX.
6. Create an array y with values [950, 1300, 800, 1000, 1300].
7. Multiply the inverse of XTX with the transpose of X, and then multiply the result by y. Call the result w.
8. What's the value of the last element of w?

> Note: You just implemented linear regression. We'll talk about it in the next lesson.

    -1.4812
    0.001
    5.6992
    23.1233

In [91]:
x = df[df["ocean_proximity"] == "ISLAND"] \
    .filter(items=["housing_median_age", "total_rooms", "total_bedrooms"]) \
    .to_numpy()

In [92]:
type(x)

numpy.ndarray

In [93]:
x.shape

(5, 3)

In [94]:
x_transpose = np.transpose(x)

In [96]:
xtx = np.dot(x_transpose, x)

In [97]:
xtx_inv = np.linalg.inv(xtx)

In [98]:
xtx_inv

array([[ 9.19403586e-04, -3.66412216e-05,  5.43072261e-05],
       [-3.66412216e-05,  8.23303633e-06, -2.77534485e-05],
       [ 5.43072261e-05, -2.77534485e-05,  1.00891325e-04]])

In [100]:
xtx_inv.shape

(3, 3)

In [99]:
y = [950, 1300, 800, 1000, 1300]

In [103]:
m1 = np.dot(xtx_inv, x_transpose)

In [104]:
m1.shape

(3, 5)

In [105]:
result = np.dot(m1, y)

In [106]:
result.shape

(3,)

In [107]:
result

array([23.12330961, -1.48124183,  5.69922946])

Answer: 5.6992